In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Github/sports-article-classification/

%pip install -r requirements_colab.txt
%pip install "ray[air]"
%pip install "ray[tune]"
%pip install -e .
%pip install transformers
%pip install datasets
%pip install slackclient
%pip install doccano-client
%cd src/modelling

/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/Github/sports-article-classification (from -r requirements_colab.txt (line 13))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.0 MB/s et

In [1]:
%cd ../..
%pip install -e .


/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///
ERROR: file:/// does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [45]:
from src.data.preprocessing import DataPreprocessorHelland
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import ray.data
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from src.slack_alert.sofus_alert import sofus_alert
from huggingface_hub import login
import configparser
from src.utils import get_project_root
from src.data.dataset import get_training_data
from ray.tune import JupyterNotebookReporter
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from transformers import EvalPrediction
import torch

In [46]:
%cd src/modelling

[Errno 2] No such file or directory: 'src/modelling'
/content/gdrive/MyDrive/Github/sports-article-classification/src/modelling


In [47]:
data = get_training_data()
data = data.loc[:, ['text', 'label']]
data_preprocessor = DataPreprocessorHelland(data['text'])
data_preprocessor.make_lower_cased()
data_preprocessor.remove_extra_spaces()
data['text'] = data_preprocessor.text_series
one_hot = MultiLabelBinarizer()
one_hot_label = one_hot.fit_transform(data['label'])
one_hot_label = [list(map(float, x)) for x in one_hot_label]
data['label'] = pd.Series(list(one_hot_label))
data

,text,label
0,premier league valgte å offentliggjøre tre fly...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1,romelu lukaku har uttalt at han er misfornøyd ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"– la oss være ærlige. jeg liker det ikke, for ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,manchester united har fått mye kritikk etter å...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,her er torsdagens oddstips!,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...
2004,marko gruljic fra red star belgrade 6. januar ...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2005,tv 2s fotballkommentator øyvind alsaker mener ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2006,de fire beste i premier league får spille i ch...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2007,"– virgil van dijk, alisson becker og mohamed s...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [48]:
train_split, validation_split = train_test_split(data, test_size=0.2, random_state=100)

In [49]:
train_dataset = Dataset.from_pandas(train_split)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = Dataset.from_pandas(validation_split)
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])
split_dict = {'train': train_dataset, 'validation': validation_dataset}
datasets = DatasetDict(split_dict)
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1607
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 402
    })
})

In [50]:
model_checkpoint = 'NbAiLab/nb-bert-large'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, model_max_length=256)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/config.json
Model config BertConfig {
  "_name_or_path": "NbAiLab/nb-bert-large",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/vocab.txt
loading file tokenizer.json from cache at None
l

In [51]:
def preprocess_function(examples):

    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256)

encoded_dataset = datasets.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [62]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    score_report = classification_report(y_true,y_pred,
    output_dict=True,
    target_names=one_hot.classes_
)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               'f1_classwise': f1_classwise.tolist(),
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [63]:
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=22, problem_type="multi_label_classification")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
   

In [64]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=22, problem_type="multi_label_classification", return_dict=True)

In [65]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token = config['huggingface']['TOKEN']

login(token=token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [66]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 393 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50000, 1024)
bert.embeddings.position_embeddings.weight               (512, 1024)
bert.embeddings.token_type_embeddings.weight               (2, 1024)
bert.embeddings.LayerNorm.weight                             (1024,)
bert.embeddings.LayerNorm.bias                               (1024,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.query.bias               (1024,)
bert.encoder.layer.0.attention.self.key.weight          (1024, 1024)
bert.encoder.layer.0.attention.self.key.bias                 (1024,)
bert.encoder.layer.0.attention.self.value.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.value.bias               (1024,)
bert.encoder.layer.0.attention.output.dense.weight      (1024, 1024)
bert.encoder.layer.0.attention.output.dense.bias             (

In [67]:
metric_name = 'accuracy'
model_name = model_checkpoint.split('/')[-1]
task = 'multi_label_classification'
#batch_size = 16

args = TrainingArguments(
    output_dir = f'experiment1-{model_name}-finetuned-{task}',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    seed = 100,
#    learning_rate=2e-5,
#    per_device_train_batch_size=batch_size,
#    per_device_eval_batch_size=batch_size,
#    num_train_epochs=5,
#    weight_decay=0.01,
   load_best_model_at_end=False,
    #metric_for_best_model=metric_name,
    #push_to_hub=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [68]:
validation_key = "validation"
trainer = Trainer(
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
   

In [69]:
def ray_hp_space(trial):
    return {
        'learning_rate': ray.tune.loguniform(1e-6, 1e-4),
        'per_device_train_batch_size': ray.tune.choice([16, 32]),
        'warmup_ratio': ray.tune.uniform(0, 0.5),
        'epochs': ray.tune.choice([3, 4, 5])
    }

In [75]:
def optimize_objective(metrics):

    #objective_metrics = ['weighted avg', '']
    objective = metrics['eval_score_report']['weighted avg']['f1-score']


    #objective_metrics = ['eval_f1_macro', 'eval_roc_auc', 'eval_accuracy']
    #objective = dict(filter(lambda item: item[0] in objective_metrics, metrics.items()))
    return objective

In [76]:
#from ray.tune import ResultGrid
bestrun = trainer.hyperparameter_search(
    direction='maximize', 
    backend='ray', 
    n_trials=1, # number of trials
    hp_space=ray_hp_space,
    progress_reporter=JupyterNotebookReporter(),
    compute_objective=optimize_objective

)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
(pid=26891) 2023-03-24 08:34:59.072010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=26891) 2023-03-24 08:34:59.072129: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
(pid=26891) 2023-03-24 08:34:59.072145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

Trial name,date,done,episodes_total,epoch,eval_accuracy,eval_f1_classwise,eval_f1_macro,eval_f1_micro,eval_loss,eval_roc_auc,eval_runtime,eval_samples_per_second,eval_score_report,eval_steps_per_second,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,objective,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
_objective_c16cb_00000,2023-03-24_08-38-35,True,,3,0.0621891,"[0.0, 0.0, 0.0, 0.0, 0.044444444444444446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.37542662116040953, 0.0, 0.0, 0.0, 0.0, 0.0]",0.019085,0.147368,0.328145,0.539892,4.8574,82.76,"{'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'Chance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20}, 'Description': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 39}, 'Garbage': {'precision': 1.0, 'recall': 0.022727272727272728, 'f1-score': 0.044444444444444446, 'support': 44}, 'Goal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 30}, 'Injury': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Link': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, 'Next game': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Odds': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Opinion': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, 'Quote': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 78}, 'Rumour': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Set-piece': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, 'Statement': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 71}, 'Statistics': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11}, 'Storytelling': {'precision': 0.5978260869565217, 'recall': 0.2736318407960199, 'f1-score': 0.37542662116040953, 'support': 201}, 'Substitution': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'Summary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 38}, 'Table': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'Transfer': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'VAR': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.448, 'recall': 0.08818897637795275, 'f1-score': 0.14736842105263157, 'support': 635}, 'macro avg': {'precision': 0.07262845849802371, 'recall': 0.013470868796513298, 'f1-score': 0.019085048436584272, 'support': 635}, 'weighted avg': {'precision': 0.25852447791852107, 'recall': 0.08818897637795274, 'f1-score': 0.12191544316346123, 'support': 635}, 'samples avg': {'precision': 0.1351575456053068, 'recall': 0.10199004975124377, 'f1-score': 0.11041586937109324, 'support': 635}}",10.499,60c5e7269f1840008c40b94ac59b44f1,"0_epochs=3,learning_rate=0.0000,per_device_train_batch_size=16,warmup_ratio=0.0684",efe32feb2e51,3,172.28.0.12,0.121915,26891,213.967,70.9215,213.967,1679647115,0,,3,c16cb_00000,0.00420141


(_objective pid=26891) 
100%|██████████| 51/51 [00:04<00:00, 11.56it/s]/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(_objective pid=26891)   _warn_prf(average, modifier, msg_start, len(result))
(_objective pid=26891) /usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
(_objective pid=26891)   _warn_prf(average, modifier, msg_start, len(result))
                                                 
100%|██████████| 51/51 [00:04<00:00, 11.56it/s]
                                               


(_objective pid=26891) {'eval_loss': 0.5231631994247437, 'eval_f1_micro': 0.13748657357679914, 'eval_f1_macro': 0.07351373544328468, 'eval_f1_classwise': [0.0, 0.028368794326241134, 0.0, 0.16981132075471697, 0.5420560747663551, 0.14102564102564102, 0.027027027027027025, 0.0, 0.04819277108433735, 0.0, 0.0, 0.09174311926605504, 0.0, 0.0, 0.2268041237113402, 0.0, 0.26819923371647514, 0.0, 0.0, 0.07407407407407408, 0.0, 0.0], 'eval_roc_auc': 0.5338486757860347, 'eval_accuracy': 0.004975124378109453, 'eval_score_report': {'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'Chance': {'precision': 0.015267175572519083, 'recall': 0.2, 'f1-score': 0.028368794326241134, 'support': 10}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20}, 'Description': {'precision': 0.13432835820895522, 'recall': 0.23076923076923078, 'f1-score': 0.16981132075471697, 'support': 39}, 'Garbage': {'precision': 0.4603174603174603, 'recall': 0.6590909090909091, 'f1

 67%|██████▋   | 202/303 [02:08<00:48,  2.07it/s]
(_objective pid=26891) 
  0%|          | 0/51 [00:00<?, ?it/s]
(_objective pid=26891) 
  6%|▌         | 3/51 [00:00<00:03, 15.65it/s]
(_objective pid=26891) 
 10%|▉         | 5/51 [00:00<00:03, 12.58it/s]
(_objective pid=26891) 
 14%|█▎        | 7/51 [00:00<00:03, 11.59it/s]
(_objective pid=26891) 
 18%|█▊        | 9/51 [00:00<00:03, 11.10it/s]
(_objective pid=26891) 
 22%|██▏       | 11/51 [00:00<00:03, 10.86it/s]
(_objective pid=26891) 
 25%|██▌       | 13/51 [00:01<00:03, 10.71it/s]
(_objective pid=26891) 
 29%|██▉       | 15/51 [00:01<00:03, 10.62it/s]
(_objective pid=26891) 
 33%|███▎      | 17/51 [00:01<00:03, 10.56it/s]
(_objective pid=26891) 
 37%|███▋      | 19/51 [00:01<00:03, 10.52it/s]
(_objective pid=26891) 
 41%|████      | 21/51 [00:01<00:02, 10.49it/s]
(_objective pid=26891) 
 45%|████▌     | 23/51 [00:02<00:02, 10.46it/s]
(_objective pid=26891) 
 49%|████▉     | 25/51 [00:02<00:02, 10.44it/s]
(_objective pid=26891) 
 53

(_objective pid=26891) {'eval_loss': 0.3678371012210846, 'eval_f1_micro': 0.09571788413098237, 'eval_f1_macro': 0.018248595376254954, 'eval_f1_classwise': [0.0, 0.0, 0.0, 0.0, 0.1276595744680851, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02380952380952381, 0.0, 0.25000000000000006, 0.0, 0.0, 0.0, 0.0, 0.0], 'eval_roc_auc': 0.5225513000423004, 'eval_accuracy': 0.0472636815920398, 'eval_score_report': {'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'Chance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20}, 'Description': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 39}, 'Garbage': {'precision': 1.0, 'recall': 0.06818181818181818, 'f1-score': 0.1276595744680851, 'support': 44}, 'Goal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 30}, 'Injury': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Link': {'pre

100%|██████████| 303/303 [03:19<00:00,  2.07it/s]
(_objective pid=26891) 
  0%|          | 0/51 [00:00<?, ?it/s]
(_objective pid=26891) 
  6%|▌         | 3/51 [00:00<00:03, 15.61it/s]
(_objective pid=26891) 
 10%|▉         | 5/51 [00:00<00:03, 12.56it/s]
(_objective pid=26891) 
 14%|█▎        | 7/51 [00:00<00:03, 11.57it/s]
(_objective pid=26891) 
 18%|█▊        | 9/51 [00:00<00:03, 11.12it/s]
(_objective pid=26891) 
 22%|██▏       | 11/51 [00:00<00:03, 10.87it/s]
(_objective pid=26891) 
 25%|██▌       | 13/51 [00:01<00:03, 10.68it/s]
(_objective pid=26891) 
 29%|██▉       | 15/51 [00:01<00:03, 10.60it/s]
(_objective pid=26891) 
 33%|███▎      | 17/51 [00:01<00:03, 10.54it/s]
(_objective pid=26891) 
 37%|███▋      | 19/51 [00:01<00:03, 10.51it/s]
(_objective pid=26891) 
 41%|████      | 21/51 [00:01<00:02, 10.48it/s]
(_objective pid=26891) 
 45%|████▌     | 23/51 [00:02<00:02, 10.47it/s]
(_objective pid=26891) 
 49%|████▉     | 25/51 [00:02<00:02, 10.47it/s]
(_objective pid=26891) 
 53

(_objective pid=26891) {'eval_loss': 0.32814502716064453, 'eval_f1_micro': 0.14736842105263157, 'eval_f1_macro': 0.019085048436584272, 'eval_f1_classwise': [0.0, 0.0, 0.0, 0.0, 0.044444444444444446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.37542662116040953, 0.0, 0.0, 0.0, 0.0, 0.0], 'eval_roc_auc': 0.5398917838400911, 'eval_accuracy': 0.06218905472636816, 'eval_score_report': {'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'Chance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20}, 'Description': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 39}, 'Garbage': {'precision': 1.0, 'recall': 0.022727272727272728, 'f1-score': 0.044444444444444446, 'support': 44}, 'Goal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 30}, 'Injury': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Link': {'precision': 

100%|██████████| 303/303 [03:33<00:00,  1.42it/s]
2023-03-24 08:38:46,316	INFO tune.py:798 -- Total run time: 230.19 seconds (229.88 seconds for the tuning loop).


In [2]:
sofus_alert()

NameError: ignored

In [78]:
bestrun

BestRun(run_id='c16cb_00000', objective=0.12191544316346123, hyperparameters={'learning_rate': 2.6315733817957475e-06, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06835329484247649, 'epochs': 3})

In [ ]:
trainer.evaluate()

In [33]:
result_grid

BestRun(run_id='49e8e_00000', objective=0.8358508836408194, hyperparameters={'learning_rate': 1.2212649231377812e-05, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.20602319920255835, 'epochs': 3})

In [34]:
one_hot.classes_

array(['Booking', 'Chance', 'Commentary', 'Description', 'Garbage',
       'Goal', 'Injury', 'Link', 'Next game', 'Odds', 'Opinion', 'Quote',
       'Rumour', 'Set-piece', 'Statement', 'Statistics', 'Storytelling',
       'Substitution', 'Summary', 'Table', 'Transfer', 'VAR'],
      dtype=object)